In [1]:
"""
출처: https://github.com/joonspk-research/generative_agents/blob/main/reverie/backend_server/persona/persona.py

페르소나 모듈
"""


import math
import sys
import datetime
import random
sys.path.append('/home/elicer/main/agents_test/backend/')

import json

import os
from global_methods import *

from persona.memory_structures.spatial_memory import *
from persona.memory_structures.associative_memory import *
from persona.memory_structures.scratch import *

from persona.prompt_template.run_gpt_prompt import *

# from persona.cognitive_modules.perceive import *
# from persona.cognitive_modules.retrieve import *
# from persona.cognitive_modules.plan import *
# from persona.cognitive_modules.reflect import *
# from persona.cognitive_modules.execute import *
# from persona.cognitive_modules.converse import *

class Persona: 
    def __init__(self, name, folder_mem_saved=False):
        # PERSONA BASE STATE 
        # <name> is the full name of the persona. This is a unique identifier for
        # the persona within Reverie. 
        self.name = name

        # PERSONA MEMORY 
        # If there is already memory in folder_mem_saved, we load that. Otherwise,
        # we create new memory instances. 
        # <s_mem> is the persona's spatial memory. 
        f_s_mem_saved = f"{folder_mem_saved}/bootstrap_memory/spatial_memory.json"
        self.s_mem = MemoryTree(f_s_mem_saved)
        # <s_mem> is the persona's associative memory. 
        f_a_mem_saved = f"{folder_mem_saved}/bootstrap_memory/associative_memory"
        self.a_mem = AssociativeMemory(f_a_mem_saved)
        # <scratch> is the persona's scratch (short term memory) space. 
        scratch_saved = f"{folder_mem_saved}/bootstrap_memory/scratch.json"
        self.scratch = Scratch(scratch_saved)
        # with open(f"/home/elicer/main/agents_test/frontend/reverie/meta.json") as json_file:  
        #     reverie_meta = json.load(json_file)

        # self.start_time = datetime.datetime.strptime(
        #                     f"{reverie_meta['start_date']}, 00:00:00",  
        #                     "%B %d, %Y, %H:%M:%S")
        # self.curr_time = datetime.datetime.strptime(reverie_meta['curr_time'], 
        #                                             "%B %d, %Y, %H:%M:%S")
        # self.sec_per_step = reverie_meta['sec_per_step']
        
        # # self.maze = Maze(reverie_meta['maze_name'])
        
        # self.step = reverie_meta['step']
        



tom = Persona('Tom', \
    folder_mem_saved = "/home/elicer/main/agents_test/frontend/persona/Tom")

# meta.json 파일을 열어서 데이터를 로드합니다.
with open('/home/elicer/main/agents_test/frontend/reverie/meta.json', 'r') as file:
    data = json.load(file)

# 불러온 데이터로 persona 인스턴스의 속성을 업데이트합니다.
tom.scratch.curr_time = data['curr_time']
# tom.scratch.curr_time = datetime.datetime.strptime(tom.curr_time, "%B %d, %Y, %H:%M:%S")



In [5]:
def create_prompt_input(persona, n_m1_activity, test_input=None):
    if test_input: return test_input
    prompt_input = []
    prompt_input += [persona.scratch.get_str_iss()]
    prompt_input += [persona.scratch.name]
    prompt_input += [persona.scratch.curr_time.strftime('%A %B %d')]
    schedule_format = ""
    for i in hour_str:
        schedule_format += f"[{persona.scratch.get_str_curr_date_str()} -- {i}]"
        schedule_format += f" Activity: [Fill in]\n"
    for i in n_m1_activity:
        schedule_format = schedule_format.replace([Fill in], i, 1)
    prompt_input += [schedule_format]
    return prompt_input

create_prompt_input(tom)


['Name: Tom\nAge: 55\nInnate traits: Honesty, Bluntness, Heartiness\nLearned traits: Tom is a blacksmith who runs a forge. He believes that the items he makes should be sold at a fair price. Although he is usually blunt, he reveals a hearty nature to those he becomes close to.\nCurrently: Tom is currently pondering how to craft fine weapons. Apart from that, he spends his time making items commissioned by customers.\nLifestyle: Tom goes to bed around 10 pm, wakes up at 6 am, has lunch around noon, and dinner around 7 pm.\nDaily plan requirement: Tom wakes up at 6 a.m., eats breakfast, and goes to the forge to prepare for work. At the forge, he usually makes items to sell and greets customers while selling the items he made in the forge. However, at lunchtime, he returns home to eat lunch and works again until 6 p.m. When 6 p.m. comes, he goes home to end his day and falls asleep around 10 p.m.\nCurrent Date: Monday November 06\n',
 'Tom',
 'Monday November 06']

In [31]:
hour_str = ["00:00 AM", "01:00 AM", "02:00 AM", "03:00 AM", "04:00 AM", 
            "05:00 AM", "06:00 AM", "07:00 AM", "08:00 AM", "09:00 AM", 
            "10:00 AM", "11:00 AM", "12:00 PM", "01:00 PM", "02:00 PM", 
            "03:00 PM", "04:00 PM", "05:00 PM", "06:00 PM", "07:00 PM",
            "08:00 PM", "09:00 PM", "10:00 PM", "11:00 PM"]
n_m1_activity = []
wake_up_hour = 6
for count, curr_hour_str in enumerate(hour_str):
    if wake_up_hour > 0:
        n_m1_activity += ["sleeping"]
        wake_up_hour -= 1


# run_local_prompt_generate_hourly_schedule(
#                             tom, 1, n_m1_activity, hour_str)[0]

In [32]:
n_m1_activity

['sleeping', 'sleeping', 'sleeping', 'sleeping', 'sleeping', 'sleeping']

In [33]:
schedule_format=""
for i in hour_str:
    schedule_format += f"[monday -- {i}]"
    schedule_format += f" Activity: [Fill in]\n"
for i in n_m1_activity:
    print(i)
    schedule_format = schedule_format.replace("[Fill in]", i, 1)

sleeping
sleeping
sleeping
sleeping
sleeping
sleeping


In [34]:
schedule_format

'[monday -- 00:00 AM] Activity: sleeping\n[monday -- 01:00 AM] Activity: sleeping\n[monday -- 02:00 AM] Activity: sleeping\n[monday -- 03:00 AM] Activity: sleeping\n[monday -- 04:00 AM] Activity: sleeping\n[monday -- 05:00 AM] Activity: sleeping\n[monday -- 06:00 AM] Activity: [Fill in]\n[monday -- 07:00 AM] Activity: [Fill in]\n[monday -- 08:00 AM] Activity: [Fill in]\n[monday -- 09:00 AM] Activity: [Fill in]\n[monday -- 10:00 AM] Activity: [Fill in]\n[monday -- 11:00 AM] Activity: [Fill in]\n[monday -- 12:00 PM] Activity: [Fill in]\n[monday -- 01:00 PM] Activity: [Fill in]\n[monday -- 02:00 PM] Activity: [Fill in]\n[monday -- 03:00 PM] Activity: [Fill in]\n[monday -- 04:00 PM] Activity: [Fill in]\n[monday -- 05:00 PM] Activity: [Fill in]\n[monday -- 06:00 PM] Activity: [Fill in]\n[monday -- 07:00 PM] Activity: [Fill in]\n[monday -- 08:00 PM] Activity: [Fill in]\n[monday -- 09:00 PM] Activity: [Fill in]\n[monday -- 10:00 PM] Activity: [Fill in]\n[monday -- 11:00 PM] Activity: [Fill in

In [26]:
schedule_format=""